In [23]:
open System
open System.Collections.Generic
open System.IO
open System.Text.RegularExpressions

let ResolutionFolder = __SOURCE_DIRECTORY__

let regexMapper find extract line =
    let finder = Regex(find, RegexOptions.Compiled)
    let extractor = Regex(extract, RegexOptions.Compiled)
    if finder.IsMatch(line) then
        seq {
            for stack in extractor.Matches(line) do
                yield stack.Value
        } |> Some
    else
        None

let (|StackLine|_|) = regexMapper @"\[\w\]" @".{3}\s?"

let (|MoveLine|_|) = regexMapper @"move \d{1,2} from \d to \d" @"\d{1,2}"

type Line =
| Stack of string seq
| Move of string seq

let mapper =
    function
    | StackLine line -> line |> Stack |> Some
    | MoveLine line -> line |> Move |> Some
    | _ -> None

type State = Map<string, string list>

let stacker (state: State) (line: string seq) =
    line
    |> Seq.mapi (fun i crate -> string (i + 1), crate)
    |> Seq.filter( fun (_,crate) -> not <| String.IsNullOrWhiteSpace(crate) )
    |> Seq.fold (fun (acc:State) (i, crate) -> 
        acc 
        |> Map.change i (function
                        | Some crates -> Some (crates @ [crate] ) 
                        | None -> Some [crate])
    ) state

let rec crateMover9000 origin dest quantity =
    match origin with
    | [] -> origin,dest
    | head :: tail -> 
        if quantity = 1 then
            (tail), ([head] @ dest)
        else
            crateMover9000 (tail) ([head] @ dest) (quantity - 1)

let mover crateMover (state:State) (move: string seq) =
    match (move |> List.ofSeq) with
    | [quantity; origin; dest] ->
        let (o,d) = crateMover (state.Item(origin)) (state.Item(dest)) (int quantity)
        state
        |> Map.change origin (function Some _ -> Some o | None -> Some o)
        |> Map.change dest (function Some _ -> Some d | None -> Some d)
    | _ -> state

let folder crateMover (state:State) (line:Line) =
    match line with
    | Stack stack ->
        stacker state stack 
    | Move move ->
        mover crateMover state move

File.ReadLines(ResolutionFolder + "/input5.txt")
|> Seq.map mapper
|> Seq.choose id
|> Seq.fold (folder crateMover9000) Map.empty
|> Map.values
|> Seq.mapi (fun i stack -> (i, stack |> Seq.tryHead))

index Item1 Item2 0 0 Value [C] 1 1 Value [M] 2 2 Value [Z]

In [28]:
let rec crateMover9001 origin dest quantity =
    match origin with
    | [] -> origin,dest
    | head :: tail -> 
        let crates = origin |> List.take quantity
        (origin |> List.skip quantity),(crates @ dest)

File.ReadLines(ResolutionFolder + "/input5.txt")
|> Seq.map mapper
|> Seq.choose id
|> Seq.fold (folder crateMover9001) Map.empty
|> Map.values
|> Seq.mapi (fun i stack -> (i, stack |> Seq.tryHead))

index Item1 Item2 0 0 Value [W] 1 1 Value [J] 2 2 Value [V] 3 3 Value [R] 4 4 Value [L] 5 5 Value [S] 6 6 Value [J] 7 7 Value [J] 8 8 Value [T]